In [1]:
file = open('New_Training_Dataset.txt')
inputs = []
targets = []
for line in file:
    arr = line.split('    ')
    if len(arr) == 1:
        arr = arr[0].split()
    inputs.append(arr[1])
    targets.append(arr[0])
# print(len(inputs))
# print(len(targets))
# print(inputs[0])
# print(targets[0])

1
1
就是,动漫,游戏,英雄,这样,技巧,游戏,如何,游戏,场景,动作,游戏,流量,梦幻,盘点,玩家,动画,游戏,峡谷,游戏,手游,推荐,游戏,游戏,盘点,那些,题材,游戏,游戏,那些,火影,忍者,对决,上线,魔兽,那些,细数,经典,手机,游戏,欢迎,动漫,角色,火影,忍者,手游,细数,动漫,那些,角色,细数,游戏,那些,让人,角色,神作,推荐,如此,盘点,游戏,盘点,魔兽,令人,热血,动漫,爱情,图片,那些,心灵,走过,动漫,经典,盘点,动漫,只想,知道,自己,惬意,事情,动漫,那些,让人,角色,动漫,智商,盘点,平台,手游,体验,英雄,三星,英雄,只想,知道,自己,惬意,事情,动漫,爱情,图片,那些,心灵,走过,动漫,王者,荣耀,英雄,盘点,那些,英雄,国产,逆天,传奇,手游,盘点,角色,热门,手游,盘点,就是,角色,那些,让人,无法,画面,细数,国产,动漫,网友,梦幻,二次元,令人,游戏,逆天,装备,装备,盘点,盘点,那些,英雄,动作,策略,玩法,盘点,游戏,推荐,玩家,游戏,盘点,智商,战争,题材,策略,手游,盘点,游戏,盘点,如何,世界,移动,出来,口袋,妖怪,怎样,梦幻,来袭,王者,荣耀,王者,腾讯,正式,亚索,攻略,这些,亚索,玩家,皮肤,场景,三星,为何,如此,欢迎,正确,出来,游戏,游戏,手游,营销,游戏,盘点,王者,王者,荣耀,腾讯,升级,银行,攻略,银行,营销,二次元,魔兽,世界,世界,画面,传奇,世界,攻略,国漫,手机,直播,玩家,亚索,正确,玩法,游戏,游戏,世界,游戏,什么,校区,手游,上天,这些,国漫,直播,到底,这些,手机,流量,召唤,移动,飞车,世界,战争,来袭,口袋,妖怪,终于,爆裂,飞车,上线,来袭,英雄,爆裂,飞车,动画,视频,平台,如此,爆裂,飞车,飞车,使命,使命,召唤,那些,虚拟,世界,那些,令人,神作,游戏,爆裂,飞车,手游,正式,来袭,升级,到底,视频,动漫,策略,这些,玩家,虚拟,巨乳,动漫,巨乳,梦想,正式,游戏,什么,玩家,皮肤,为何,网友,传奇,上天,王者,乱世,王者,这些,手游,上天,校区,峡谷,这样,领取,飞车,智商,无法,领取,梦想,怎样,体验,王者,荣耀,皮肤,英雄,自己,终于,热门,游戏,乱世,英雄,玩家,对决,热血,虚空,物语,攻略,技巧,虚空,物语,三星,英雄,推荐娱乐 游戏
娱乐 游戏


In [ ]:
words_set = []
for text in inputs:
    for word in text.split(','):
        words_set.append(word)
words_set = set(words_set)
vocab_to_int = {word:idx for idx,word in enumerate(words_set)}
int_to_vocab = {idx:word for idx,word in enumerate(words_set)}

print(len(words_set))

In [ ]:
inputs_int = []
for text in inputs:
    inputs_int.append([vocab_to_int[word] for word in text.split(',')])
print(len(inputs_int))
print(inputs_int[0])

In [ ]:
targets_set = set(targets)
targets_to_int = {word:idx for idx,word in enumerate(targets_set)}
int_to_targets = {idx:word for idx,word in enumerate(targets_set)}
targets_int = [targets_to_int[word] for word in targets]
print(len(targets_int))
print(targets_int[:10])

In [ ]:
import numpy as np
from keras.utils.np_utils import to_categorical
labels = to_categorical(np.array(targets_int))
print(labels[:10])

In [ ]:
from collections import Counter
seq_len = 400
features = np.zeros((len(inputs_int), seq_len), dtype=int)
for i, row in enumerate(inputs_int):
    counts = Counter(row)
    row = sorted(row, key=lambda x : counts[x], reverse=True)
    features[i, -len(row):] = np.array(row)[:seq_len]

In [ ]:
#获取训练集
features = np.array(features)
from sklearn.model_selection import StratifiedShuffleSplit
ss = StratifiedShuffleSplit(n_splits=1, test_size=0.2)
train_idx, val_idx = next(ss.split(features, labels))
half_val_len = int(len(val_idx)/2)
val_idx, test_idx = val_idx[:half_val_len], val_idx[half_val_len:]
#split the training data, valid data and test data
train_x, train_y = features[train_idx], labels[train_idx]
val_x, val_y = features[val_idx], labels[val_idx]
test_x, test_y = features[test_idx], labels[test_idx]

print(len(train_x) , len(train_y))
print(len(val_x) , len(val_y))
print(len(test_x) , len(test_y))

In [ ]:
from keras.initializers import TruncatedNormal
from keras.models import Sequential, Model
from keras.layers import concatenate,Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Embedding
def text_cnn(maxlen = 400, max_features = len(words_set), embed_size = 300):
    comment_seq = Input(shape=[maxlen],name='x_seq')
    emb_comment = Embedding(max_features,embed_size,embeddings_initializer=TruncatedNormal(mean=0.0, stddev=0.05, seed=None))(comment_seq)
    #conv layers
    convs = []
    filter_sizes = [2,3,4,5]
    for fsz in filter_sizes:
        l_conv = Conv1D(filters=100, kernel_size = fsz, activation='tanh')(emb_comment)
        l_pool = MaxPooling1D(maxlen-fsz+1)(l_conv)
        l_pool = Flatten()(l_pool)
        convs.append(l_pool)
    merge = concatenate(convs, axis=1)
    out = Dropout(0.5)(merge)
    output = Dense(256,activation='relu')(out)
    output = Dense(43, activation='softmax')(output)
    model = Model([comment_seq],output)
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    return model

In [ ]:
model = text_cnn()

In [ ]:
from keras.callbacks import ModelCheckpoint   
# train the model
checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose=1, 
                               save_best_only=True)
history = model.fit(train_x, train_y, batch_size=500, epochs=5,
          validation_data=(val_x, val_y), callbacks=[checkpointer], 
          verbose=1, shuffle=True)
model.save('model_textcnn_detection.h5') 

In [ ]:
#model testing
model.load_weights('model.weights.best.hdf5')
score = model.evaluate(test_x, test_y, verbose=1)
print('\n', 'Test accuracy:', score[1])

In [ ]:
from keras.models import load_model
text_model = load_model('model_textcnn_detection.h5')
text_model.load_weights('model.weights.best.hdf5')
score = text_model.evaluate(test_x, test_y, verbose=1)
print('\n', 'Test accuracy:', score[1])

In [ ]:
import pickle
import numpy as np
from keras.models import load_model
from collections import Counter

class Predict_labels:
    def __init__(self):
        #读取模型
        self.model = load_model('model_textcnn_detection.h5')
        self.model.load_weights('model.weights.best.hdf5')
        #读取int_to_targets词典:{每个标签的one-hot编码的位置：标签名称}
        with open('int_to_targets.pickle', 'rb') as handle:
            self.int_to_targets = pickle.load(handle)
        #读取vocab_to_int词典: {每个单词的hash code: 每个单词}
        with open('vocab_to_int.pickle', 'rb') as handle:
            self.vocab_to_int = pickle.load(handle)
        stop_words_file = open('stopword.txt')
        self.stopwords = set([item.strip('\n') for item in stop_words_file])
        self.special_sign = set(['',' ','\u3000','｜','一','…','丨','←'])
    def make_classification(self,text):
        #将输入text转换为数字数组
        text_int = [self.vocab_to_int[word] if word in self.vocab_to_int else 0 for word in text.split()]
        seq_len = 400
        #提取前400个频率较高的词，若长度不足400，在前面补0
        feature = np.zeros(seq_len, dtype=int)
        counts = Counter(text_int)
        row = sorted(text_int, key = lambda x : counts[x], reverse=True)
        if(len(row) == 0):
            return 'none'
        feature[-len(row):] = np.array(row)[:seq_len]
        #模型预测标签并返回分类
        label = self.model.predict(feature[None,:])
        return self.int_to_targets[np.argmax(label)]
p = Predict_labels()
text_input = '牛初乳 自然 生命 阳光 牛初乳 临床 研究 证实 新西兰 牛初乳 重点 妈妈 妈妈 母乳 喂养 抵抗力 婴幼儿 老人 人群 人群 解读 生命 阳光 牛初乳 生命 爱心 营养 功能 生命 阳光 牛初乳 市场 营养 婴幼儿 健康 问题 牛初乳 生产 选择 牛初乳 奶源 生命 阳光 牛初乳 免疫 球蛋白 免疫 蛋白 免疫 球蛋白 提高 免疫力 预防 手足口 婴幼儿 用药 提高 免疫力 根本 免疫力 生命 阳光 金奇仕 营养 健康 呼吸道 疾病 免疫 功能 自然 母乳 喂养 提高 孩子 免疫力 免疫 营养 提高 免疫力 免疫 系统 问题 婴幼儿 原因 营养 均衡 提高 免疫力 适当 生命 阳光 牛初乳 免疫 营养 提高 免疫力 无惧 挑战 老人 免疫力 饮食 均衡 营养 适当 补充 免疫 营养 经常 睡眠 合理 健康 呼吸道 疾病 免疫 功能 市场 牛初乳 新西兰 生命 阳光 牛初乳 提高 免疫力 宝宝 秋季 注意 预防 秋季 疾病 感冒 护理 合理 用药 准妈妈 秋季 护理 注意 事项 母乳 喂养 注意 饮食 病毒 感染 病毒 疫苗 提高 免疫力 调节 预防 宝宝 秋季 腹泻 研究 证实 生命 阳光 牛初乳 可以 预防 腹泻 秋季 增强 免疫力 宝宝 免疫 功能 系统 抗生素 秋季 腹泻 原因 生命 阳光 牛初乳 宝宝 秋季 生命 阳光 牛初乳 调节 人体 免疫力 预防 秋季 呼吸道 感染 免疫 营养 重点 疫苗 家庭 宝宝 免疫力 宝宝 宝宝 抗生素 适当 补充 生命 阳光 牛初乳 免疫 营养 提高 宝宝 免疫力 睡眠 免疫力 提高 天然 于宝宝 吸收 天然 吸收 因子 帮助 宝宝 吸收 天然 因子 蛋白 帮助 锁住 宝宝 经常 补钙 还是 临床 研究 证实 生命 阳光 可以 奶源 吸收 于宝宝 对于 宝宝 补钙 生命 阳光 宝宝 选择 补钙 产品 关键 对于 补钙 可以 补钙 关键 生命 阳光 补钙 锁住 孩子 补钙 补充 金奇仕 补钙 宝宝 婴幼儿 补钙 补钙 补钙 产品 关键 准妈妈 补钙 增强 抵抗力 根本 牛初乳 益生元 免疫 体质 无惧 挑战 还是 牛初乳 益生元 免疫 体质 无惧 挑战 预防 预防 宝宝 孩子 体质 健康 无惧 生命 阳光 营养 牛初乳 春季 宝宝 提高 抵抗力 提高 抗病 春季 宝宝 春季 抗病 春季 宝宝 注意 补钙 宝宝 感冒 护理 春季 预防 宝宝 春季 感冒 宝宝 补充 生命 阳光 帮助 宝宝 解读 提高 老人 健康 提高 人体 抵抗力 牛初乳 春季 宝宝 春季 春季 疾病 准妈妈 注意 提高 抵抗力 预防 提高 抵抗力 关键 宝宝 提高 抵抗力 关键 补钙 爱心 奶粉 托起 健康 生命 爱心 奶粉 托起 健康 生命 健康 生命 婴幼儿 生命 阳光 产品 生产 工厂 首批 通过 认证 生命 阳光 产品 生产 工厂 首批 通过 认证 手足口 宝宝 预防 关键 提高 免疫力 疾病 妈妈 补钙 产品 质量 生命 阳光 牛初乳 质量 预防 宝宝 孩子 必须 注意 事项 必须 家庭 如何 新西兰 生命 阳光 奶粉 解读 奶粉 如何 选择 孩子'
print(p.make_classification(text_input))

In [ ]:
from gensim import corpora  
from gensim.models import LdaModel  
from gensim.corpora import Dictionary 
from gensim import models
lda_model = models.LdaModel.load('ldamodel_45.model')
model_id2word = models.LdaModel.load('ldamodel_45.model.id2word')

In [ ]:
filepath = 'testing_data/part-r-000'
file_arr = []
for i in range(10,21):
    file_arr.append(open(filepath + str(i)))
texts = []
for file in file_arr:
    for line in file:
        texts.append(line.split('	')[1])
print(len(texts))

In [ ]:
category_dic = {0:'美文 ', 1:'博彩', 2: '医疗健康 生理', 3: '购物 线上购物', 4: '购物 通用',5:'购物 汽车',6:'酒店餐饮 酒店婚庆', 7:'资讯 娱乐八卦', 8:'创业营销,励志鸡汤',9:'旅游'
      ,10:'娱乐 游戏影视',11:'资讯 投资理财', 12: '酒店餐饮 美食推荐', 13: '生活服务 金融理财', 14:'医疗健康 保健养生', 15:'资讯 投资理财', 16:'资讯 社会', 17:'教育 课外培训',18:'购物 电子设备', 19:'企业管理', 
       20:'生活服务 婚纱摄影', 21:'教育 儿童教育', 22:'资讯 军事', 23:'查询 中奖信息', 24:'资讯 会记税收',25:'生活服务 宗教',26:'资讯 党政',27:'医疗健康 生理',28:'教育 中小学',29:'情感生活 婚姻生活',30:'情感生活 男女八卦',
               31:'教育 留学',32:'购物 汽车',33:'医疗健康 母婴', 34:'资讯 通用',35:'娱乐 搞笑段子',36:'查询 通讯',37:'资讯 投资理财',38:'医疗健康 美容保健',39:'生活服务 家具装修',40:'医疗健康 生理卫生',41:'医疗健康 减肥健身',
               42:'资讯 房地产',43:'资讯 社会民生',44:'购物 线上购物'}

In [ ]:
from tqdm import *
category_text_dic = {}
for i in tqdm(range(len(texts))):
    text = texts[i]
    label_cnn = p.make_classification(text)
    doc_bow = model_id2word.doc2bow(text.split())
    doc_lda = lda_model[doc_bow]
    doc_lda.sort(key=lambda x : -x[1])
    topic = doc_lda[0][0]
    if doc_lda[0][0] == 40 and doc_lda[0][1] == 30: #区分情趣和生理标签
        topic = 30
    str_temp = category_dic[topic]
    if doc_lda[0][0] == 3:
        word_set = set(['书画','赏析','书法','艺术','文化'])
        for word in word_set:
            str_temp = '艺术鉴赏'
    if doc_lda[0][0] == 41:
        word_set = set(['搭配','裙子','外套','女装','男装','单品','衬衫','潮流','童装','街拍'])
        for word in word_set:
            str_temp = '购物 流行时装'
    if doc_lda[0][0] == 19:
        word_set = set(['科技','互联网','智能','数据'])
        for word in word_set:
            str_temp = '互联网'
    if(doc_lda[0][1] > 0.8): 
        if str_temp not in category_text_dic:
            category_text_dic[str_temp] = [text]
        else:
            category_text_dic[str_temp].append(text)
    else:
        if label_cnn not in category_text_dic:
            category_text_dic[label_cnn] = [text]
        else:
            category_text_dic[label_cnn].append(text)

In [ ]:
for key in category_text_dic:
    print(key, len(category_text_dic[key]))

In [ ]:
import pandas as pd
labels = []
contents = []
for key in category_text_dic:
    if len(category_text_dic[key]) > 30:
        for item in category_text_dic[key][:30]:
            if key == '医疗健康 生理':
                labels.append('医疗健康 生理卫生')
            else:
                labels.append(key)
            contents.append(item)
    else:
        for item in category_text_dic[key]:
            labels.append(key)
            contents.append(item)
print(len(labels))
print(len(contents))

In [ ]:
dataframe = pd.DataFrame({'Titles': contents, 'Labels': labels})
dataframe.to_csv('test_account_classify.csv',index=False,sep=',',encoding='utf-8-sig')
print('finished')

In [ ]:
with open("test_account_classify.txt","w") as f:
        for i in range(len(labels)):
            f.write(labels[i] + '   ' + contents[i])
print('finished')

In [ ]:
import xlwt

workbook = xlwt.Workbook(encoding='utf-8')
booksheet = workbook.add_sheet('Sheet 1', cell_overwrite_ok=True)
for i in range(1290):
    booksheet.write(i,0,labels[i])
    booksheet.write(i,1,contents[i])
workbook.save('test_account_classify.xls')
print('finished')

In [3]:
import random
a = [1,2,3]
random.shuffle (a)
print(a)

[3, 2, 1]
